In [4]:
from sklearn.cluster import DBSCAN

In [21]:
from sklearn.preprocessing import StandardScaler
from trackml.dataset import load_event
import pandas as pd
import numpy as np
import glob

In [24]:
train = np.unique([p.split('-')[0] for p in sorted(glob.glob('C:/Users/sumas/Downloads/train_sample/train_100_events/**'))])
test = np.unique([p.split('-')[0] for p in sorted(glob.glob('C:/Users/sumas/Downloads/test/test/**'))])
det = pd.read_csv('C:/Users/sumas/Downloads/detectors/detectors.csv')
sub = pd.read_csv('C:/Users/sumas/Downloads/sample_submission.csv/sample_submission.csv')
print(len(train), len(test), len(det), len(sub))
data.head()

100 125 18728 13741466


,hit_id,x,y,z,volume_id,layer_id,module_id
0,1,-64.4099,-7.163700,-1502.5,7,2,1
1,2,-55.3361,0.635342,-1502.5,7,2,1
2,3,-83.8305,-1.143010,-1502.5,7,2,1
3,4,-96.1091,-8.241030,-1502.5,7,2,1
4,5,-62.6736,-9.371200,-1502.5,7,2,1


In [25]:
stscaler = StandardScaler()
dbscan = DBSCAN(eps=0.00715, min_samples=1, algorithm='auto', n_jobs=-1)

In [31]:
df_test = []
for e in test:
    hits, cells = load_event(e, parts=['hits', 'cells'])
    hits['event_id'] = int(e[-9:])
    cells = cells.groupby(by=['hit_id'])['ch0', 'ch1', 'value'].agg(['mean']).reset_index()
    cells.columns = ['hit_id', 'ch0', 'ch1', 'value']
    hits = pd.merge(hits, cells, how='left', on='hit_id')
    col = [c for c in hits.columns if c not in ['event_id', 'hit_id', 'particle_id']]
     #https://www.kaggle.com/mikhailhushchyn/dbscan-benchmark
    x = hits.x.values
    y = hits.y.values
    z = hits.z.values
    r = np.sqrt(x**2 + y**2 + z**2)
    hits['x2'] = x/r
    hits['y2'] = y/r
    r = np.sqrt(x**2 + y**2)
    hits['z2'] = z/r
    hits['particle_id'] = dbscan.fit_predict(stscaler.fit_transform(hits[['x2', 'y2', 'z2']].values))
    df_test.append(hits[['event_id','hit_id','particle_id']].copy())
    
df_test = pd.concat(df_test, ignore_index=True)

sub = pd.merge(sub, df_test, how='left', on=['event_id','hit_id'])
sub['track_id'] = sub['particle_id'] + 1
sub[['event_id','hit_id','track_id']].to_csv('submission-001.csv', index=False)   

,hit_id,x,y,z,volume_id,layer_id,module_id,event_id,ch0,ch1,value,x2,y2,z2,particle_id
0,1,-58.966499,-1.40579,-1502.5,7,2,1,124,103.0,513.5,0.148556,-0.039215,-0.000935,-25.473331,0
1,2,-57.125702,-3.86896,-1502.5,7,2,1,124,155.0,484.0,0.303995,-0.037993,-0.002573,-26.241531,1
2,3,-63.967899,-10.27550,-1502.5,7,2,1,124,272.0,614.5,0.158574,-0.042535,-0.006833,-23.191042,2
3,4,-35.527000,-8.73935,-1502.5,7,2,1,124,286.0,108.0,0.319781,-0.023638,-0.005815,-41.067490,3
4,5,-76.471603,-7.54816,-1502.5,7,2,1,124,198.0,832.0,0.285553,-0.050830,-0.005017,-19.552797,4


In [ ]:
 #https://www.kaggle.com/mikhailhushchyn/dbscan-benchmark
    x = hits.x.values
    y = hits.y.values
    z = hits.z.values
    r = np.sqrt(x**2 + y**2 + z**2)
    hits['x2'] = x/r
    hits['y2'] = y/r
    r = np.sqrt(x**2 + y**2)
    hits['z2'] = z/r
    hits['particle_id'] = dbscan.fit_predict(stscaler.fit_transform(hits[['x2', 'y2', 'z2']].values))
    df_test.append(hits[['event_id','hit_id','particle_id']].copy())
    print(e, len(hits['particle_id'].unique()))
    #break
    
df_test = pd.concat(df_test, ignore_index=True)

sub = pd.merge(sub, df_test, how='left', on=['event_id','hit_id'])
sub['track_id'] = sub['particle_id'] + 1
sub[['event_id','hit_id','track_id']].to_csv('submission-001.csv', index=False)